# Generación de archivos parquet SIVIGILA

## Carga de los datos

Este jupyter notebook esta generando con todos los datos de SIVIGILA (2008 - 2022), estos datos tienen errores en la fuente inicial de los parquetes que se cargan. ( Columnas corridas, años por fuera del rango esperado, entre otros notificados en la fase de exploración)  

In [65]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasSIVIGILA_Todos").getOrCreate()

In [66]:
#spark.stop()

In [67]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/sivigila/sivigila_parquet_*/")

df=df1_parquet

In [68]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 7815255
Columnas del DataFrame unido: ['PersonaBasicaID', 'Evento', 'AñoEpidemiologico', 'SemanaEpidemilogica', 'Prestador', 'CodigoAdministradora', 'Administradora', 'Edad', 'UnidadMedidaEdad', 'Sexo', 'TipoRegimen', 'codigoOcupacion', 'Ocupacion', 'Etnia', 'ClasificacioninicialCaso', 'PacienteHospitalizado', 'SeguimientoFinalCaso', 'FechaNotificacion', 'FechaConsulta', 'FechaInicioSINTOmas', 'FechaHospitalizacion', 'FechaDefuncion', 'FechaNcto', 'FechaAjuste', 'GP_Discapacidad', 'GP_Despazados', 'GP_Migrantes', 'GP_Carcelarios', 'GP_Gestantes', 'GP_Indigenas', 'GP_PoblacionICBF', 'GP_MadresComunitarias', 'GP_Desmovilizados', 'GP_Psiquia', 'GP_VictimasViolencia', 'GP_otros', 'codigoPaisOcurrencia', 'PaisOcurrencia', 'codigoDepartamentoOcurrencia', 'DepartamentoOcurrencia', 'codigoMunicipioOcurrencia', 'MunicipioOcurrencia', 'codigoDepartamentoNotificacion', 'DepartamentoNotificacion', 'codigoMunicipioNotificacion', 'MunicipioNotificacion', 'co

In [69]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 (128.343)
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 7815255


In [70]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- PersonaBasicaID: string (nullable = true)
 |-- Evento: string (nullable = true)
 |-- AñoEpidemiologico: string (nullable = true)
 |-- SemanaEpidemilogica: string (nullable = true)
 |-- Prestador: string (nullable = true)
 |-- CodigoAdministradora: string (nullable = true)
 |-- Administradora: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- UnidadMedidaEdad: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- TipoRegimen: string (nullable = true)
 |-- codigoOcupacion: string (nullable = true)
 |-- Ocupacion: string (nullable = true)
 |-- Etnia: string (nullable = true)
 |-- ClasificacioninicialCaso: string (nullable = true)
 |-- PacienteHospitalizado: string (nullable = true)
 |-- SeguimientoFinalCaso: string (nullable = true)
 |-- FechaNotificacion: string (nullable = true)
 |-- FechaConsulta: string (nullable = true)
 |-- FechaInicioSINTOmas: string (nullable = true)
 |-- FechaHospitalizacion: string (nullable = true)
 |-- FechaDefuncion: stri

# Empieza el proceso de generación de los datos agrupados por atenciones

In [71]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'PersonaBasicaID','FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion'
    
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaBasicaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaBasicaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaBasicaID').agg(
    F.first('FechaInicioSINTOmas').alias('FechaInicioSINTOmas'),
    F.first('Edad').alias('Edad'),
    F.first('Sexo').alias('Sexo'),
    F.first('TipoRegimen').alias('TipoRegimen'),
    F.first('Evento').alias('Evento'),
    F.first('Area').alias('Area'),
    F.first('PacienteHospitalizado').alias('PacienteHospitalizado'),
    F.first('CondicionFinal').alias('CondicionFinal'),
    F.first('DiagnosticoDefuncion').alias('DiagnosticoDefuncion'), 
   
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------------+-------------------+----------+---------+----------------+--------------------+--------------+---------------------+--------------+--------------------+---------------+
|PersonaBasicaID|FechaInicioSINTOmas|      Edad|     Sexo|     TipoRegimen|              Evento|          Area|PacienteHospitalizado|CondicionFinal|DiagnosticoDefuncion|personaid_count|
+---------------+-------------------+----------+---------+----------------+--------------------+--------------+---------------------+--------------+--------------------+---------------+
|      100003768|           20110609|De 21 años| FEMENINO|  S - SUBSIDIADO|360 - INTOXICACIÓ...|         05101|                   SI|CIUDAD BOLÍVAR|        1 - VIVO (A)|              1|
|      100004206|           20210601|De 29 años|MASCULINO|  S - SUBSIDIADO|850 - VIH/SIDA/MO...|CENTRO POBLADO|                   NO|  1 - VIVO (A)|    0 - NO REPORTADO|              1|
|      100004382|           20131020| De 01 año|MASCULINO|  S - SUBSID

+---------------+-------------------+----------+---------+----------------+--------------------+--------------+---------------------+--------------+--------------------+---------------+
|PersonaBasicaID|FechaInicioSINTOmas|      Edad|     Sexo|     TipoRegimen|              Evento|          Area|PacienteHospitalizado|CondicionFinal|DiagnosticoDefuncion|personaid_count|
+---------------+-------------------+----------+---------+----------------+--------------------+--------------+---------------------+--------------+--------------------+---------------+
|      100003768|           20110609|De 21 años| FEMENINO|  S - SUBSIDIADO|360 - INTOXICACIÓ...|         05101|                   SI|CIUDAD BOLÍVAR|        1 - VIVO (A)|              1|
|      100004206|           20210601|De 29 años|MASCULINO|  S - SUBSIDIADO|850 - VIH/SIDA/MO...|CENTRO POBLADO|                   NO|  1 - VIVO (A)|    0 - NO REPORTADO|              1|
|      100004382|           20131020| De 01 año|MASCULINO|  S - SUBSID

In [72]:
# Contar el número de registros - me da satisfactoriamente la cantidad de registros en cada año ( 97.416)
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 6082620


In [73]:
 df_final =grouped_df


In [74]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 6082620


In [75]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------------+-------------------+----------+---------+----------------+--------------------+--------------+---------------------+--------------+--------------------+---------------+
|PersonaBasicaID|FechaInicioSINTOmas|      Edad|     Sexo|     TipoRegimen|              Evento|          Area|PacienteHospitalizado|CondicionFinal|DiagnosticoDefuncion|personaid_count|
+---------------+-------------------+----------+---------+----------------+--------------------+--------------+---------------------+--------------+--------------------+---------------+
|      100003768|           20110609|De 21 años| FEMENINO|  S - SUBSIDIADO|360 - INTOXICACIÓ...|         05101|                   SI|CIUDAD BOLÍVAR|        1 - VIVO (A)|              1|
|      100004206|           20210601|De 29 años|MASCULINO|  S - SUBSIDIADO|850 - VIH/SIDA/MO...|CENTRO POBLADO|                   NO|  1 - VIVO (A)|    0 - NO REPORTADO|              1|
|      100004382|           20131020| De 01 año|MASCULINO|  S - SUBSID

Se notifico que esta muestra tenia errores desde la fase de exploración, no se corrigieron... 

# Empieza el proceso de generación de tablas SIVIGILA

# Pagina 1 - Power BI

In [76]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

## 1. Agrupado por año, mes, Edad, Sexo y TipoRegimen

In [77]:
# Selecciona la columna 'FechaRegistro' y muestra sus datos
#df2.select('FechaInicioSINTOmas').show(100, truncate=False)

In [78]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad, sexo y tipo de régimen
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'TipoRegimen').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+----------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|     TipoRegimen|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+----------------+------------------+--------------------+
|2011|  8| De 08 años|MASCULINO|  S - SUBSIDIADO|               125|                 138|
|2020|  3| De 25 años| FEMENINO|  S - SUBSIDIADO|               168|                 214|
|2019|  9| De 24 años| FEMENINO|C - CONTRIBUTIVO|               256|                 302|
|2022|  5| De 16 años| FEMENINO|  S - SUBSIDIADO|               285|                 432|
|2022|  8| De 26 años| FEMENINO|C - CONTRIBUTIVO|               159|                 186|
|2019|  1| De 58 años| FEMENINO|  S - SUBSIDIADO|                64|                  72|
|2017|  9| De 21 años| FEMENINO|  S - SUBSIDIADO|               225|                 267|
|2018|  2| De 20 años| FEMENINO|C - CONTRIBUTIVO|               177|                 208|
|2019| 11|

In [79]:
#se escribe el parquet tabla 1- Agrupado por año, mes, Edad, sexo y tipo de régimen
result_df.write.parquet("hdfs:/analyticsdata/1.SIVIGILA_Tabla_anio_mes_Edad_sexo_TipoRegimen_Todos.parquet") 

# 2. Agrupado por año, mes, Edad, Sexo y Evento

In [80]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y Evento
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'Evento').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+---------+--------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|              Evento|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+--------------------+------------------+--------------------+
|2021| 10| De 09 años| FEMENINO|300 - AGRESIONES ...|               112|                 117|
|2020|  5| De 07 años|MASCULINO|300 - AGRESIONES ...|                75|                  85|
|2015|  2| De 24 años|MASCULINO|300 - AGRESIONES ...|                52|                  57|
|2012| 11|De 04 meses| FEMENINO|831 - VARICELA IN...|                16|                  17|
|2020|  2| De 24 años| FEMENINO|549 - MORBILIDAD ...|                68|                  75|
|2011|  4| De 03 años| FEMENINO|831 - VARICELA IN...|               274|                 286|
|2011| 11| De 03 años|MASCULINO|300 - AGRESIONES ...|                52|                  56|
|2016|  6| De 09 años|MASCULINO|300 - AGRESIONES ...|       

In [81]:
#se escribe el parquet tabla 2- Agrupado por año, mes, Edad, sexo y Evento
result_df.write.parquet("hdfs:/analyticsdata/2.SIVIGILA_Tabla_anio_mes_Edad_sexo_Evento_Todos.parquet") 

# 3.  Agrupado por año, mes, Edad, Sexo y Area

In [82]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y Area
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'Area').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+---------+-------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|               Area|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+-------------------+------------------+--------------------+
|2015|  7| De 21 años|MASCULINO|              70702|                 1|                   1|
|2012|  9|De 07 meses|MASCULINO|              73555|                 1|                   1|
|2020|  5| De 28 años|MASCULINO|              05642|                 1|                   1|
|2022| 12| De 35 años| FEMENINO|           CABECERA|                48|                  53|
|2018|  7| De 10 años|MASCULINO|     CENTRO POBLADO|                 7|                   8|
|2019| 12| De 24 años|MASCULINO|              05001|                13|                  24|
|2015|  7| De 08 años| FEMENINO|              08433|                 1|                   1|
|2018|  3| De 22 años| FEMENINO|              25473|                 2

In [83]:
#se escribe el parquet tabla 3- Agrupado por año, mes, Edad, sexo y Area
result_df.write.parquet("hdfs:/analyticsdata/3.SIVIGILA_Tabla_anio_mes_Edad_sexo_Area_Todos.parquet") 

# 4.  Agrupado por año, mes, Edad, Sexo y PacienteHospitalizado

In [84]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y PacienteHospitalizado
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'PacienteHospitalizado').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+---------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|PacienteHospitalizado|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+---------------------+------------------+--------------------+
|2017|  8| De 24 años|MASCULINO|                   NO|               231|                 255|
|2022|  9| De 10 años| FEMENINO|                   NO|               262|                 307|
|2014|  8| De 23 años| FEMENINO|                   SI|               124|                 143|
|2016|  3| De 05 años|MASCULINO|                   NO|               485|                 548|
|2021|  1| De 38 años| FEMENINO|                   NO|                86|                  99|
|2017| 10| De 13 años| FEMENINO|                   NO|               210|                 247|
|2014| 10| De 29 años| FEMENINO|                   SI|                94|                 106|
|2018|  2| De 26 años| FEMENINO|                  

In [85]:
#se escribe el parquet tabla 4- Agrupado por año, mes, Edad, sexo y PacienteHospitalizado
result_df.write.parquet("hdfs:/analyticsdata/4.SIVIGILA_Tabla_anio_mes_Edad_sexo_PacienteHospitalizado_Todos.parquet") 

# 5.  Agrupado por año, mes, Edad, Sexo y CondicionFinal

In [86]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y CondicionFinal
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'CondicionFinal').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+--------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|      CondicionFinal|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+--------------------+------------------+--------------------+
|2013|  8| De 17 años|MASCULINO|                CALI|                13|                  15|
|2020|  8| De 26 años|MASCULINO|            MEDELLÍN|                10|                  10|
|2011|  5| De 05 años| FEMENINO|            MEDELLÍN|                16|                  17|
|2017|  4| De 22 años|MASCULINO|              LA PAZ|                 1|                   1|
|2016|  3| De 24 años|MASCULINO|EL CARMEN DE BOLÍVAR|                 1|                   1|
|2015|  4| De 08 años|MASCULINO|        1 - VIVO (A)|                99|                 109|
|2019| 11| De 24 años|MASCULINO|            GUARANDA|                 1|                   3|
|2021| 10| De 07 años|MASCULINO|              MANATÍ|       

In [87]:
#se escribe el parquet tabla 5- Agrupado por año, mes, Edad, sexo y CondicionFinal
result_df.write.parquet("hdfs:/analyticsdata/5.SIVIGILA_Tabla_anio_mes_Edad_sexo_CondicionFinal_Todos.parquet") 

# 6.  Agrupado por año, mes, Edad, Sexo y DiagnosticoDefuncion

In [88]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y DiagnosticoDefuncion
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'DiagnosticoDefuncion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+--------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|DiagnosticoDefuncion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+--------------------+------------------+--------------------+
|2014| 11| De 02 años|MASCULINO|    0 - NO REPORTADO|               452|                 498|
|2009|  7| De 07 años| FEMENINO|        1 - VIVO (A)|               226|                 238|
|2013|  6| De 18 años|MASCULINO|        1 - VIVO (A)|               241|                 267|
|2011|  9| De 06 años| FEMENINO|        1 - VIVO (A)|               418|                 446|
|2020|  2| De 24 años|MASCULINO|        1 - VIVO (A)|               272|                 335|
|2011|  9| De 06 años|MASCULINO|        1 - VIVO (A)|               433|                 474|
|2014| 11| De 19 años| FEMENINO|      2 - MUERTO (A)|                36|                  45|
|2015|  4| De 08 años|MASCULINO|        1 - VIVO (A)|       

In [89]:
#se escribe el parquet tabla 6- Agrupado por año, mes, Edad, sexo y DiagnosticoDefuncion
result_df.write.parquet("hdfs:/analyticsdata/6.SIVIGILA_Tabla_anio_mes_Edad_sexo_DiagnosticoDefuncion_Todos.parquet")

In [93]:
# Leer el archivo .parquet y se guarda en un DataFrame ( para probar que quedaron guardados)
df_leer = spark.read.parquet("hdfs:/analyticsdata/5.SIVIGILA_Tabla_anio_mes_Edad_sexo_CondicionFinal_Todos.parquet")

# Mostrar el contenido del DataFrame
df_leer.show()

+----+---+----------+---------+-------------------+------------------+--------------------+
| Año|Mes|      Edad|     Sexo|     CondicionFinal|Número_de_Personas|Número_de_Atenciones|
+----+---+----------+---------+-------------------+------------------+--------------------+
|2021|  9|De 33 años| FEMENINO|        Bogotá D.C.|                21|                  23|
|2016|  7|De 22 años| FEMENINO|             SOACHA|                 5|                   5|
|2014|  9| De 01 año|MASCULINO|       1 - VIVO (A)|               419|                 457|
|2013|  6| De 01 mes|MASCULINO|       1 - VIVO (A)|                54|                  54|
|2016|  3|De 02 años|MASCULINO|             CÚCUTA|                17|                  23|
|2013|  7|De 06 años|MASCULINO|             SOACHA|                 3|                   3|
|2022|  2|De 17 años|MASCULINO|       1 - VIVO (A)|                96|                 121|
|2016|  8|De 60 años|MASCULINO|      FLORIDABLANCA|                 1|          

In [94]:
spark.stop()